## Predict Prices Models
> This notebook build use the information from the exploration step to perform the preprocessing and build the ML Models

### Goal
> predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable.

This notebook summarizes the whole preprocessing steps, includes feature selection based on visualizations in the preparation notebook, tests of various machine learning algorithms and a basic blended ml model. With that it is used as a basis to explore feature engineering, GridSearch to find the best hyperparameters for each machine learning algorithm, implement more advanced blending or even stacking methods to enhance the predictive sore.


### Content
- [Preprocessing](#pre)
- [Modeling](#modeling)
- [Use Models to make predictions](#pred)

### Data
- <a href='https://www.kaggle.com/c/house-prices-advanced-regression-techniques'>Link</a> to Kaggle competition 
- <a href='https://amstat.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627#.X1ZIXy337UI'>Paper</a> on the data: 


### Score of final model
- blended model with CatBoost, LGBMRegressor and Ridge Regression (without any hyperparameter tuning or feature engineering): 
    - Kaggle Score: 0.13608
    - top 44%

In [30]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ml related imports
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, train_test_split
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso
from catboost import CatBoostRegressor

# silence settingWithCopyWarning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# get the data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train.shape, test.shape

((1460, 81), (1459, 80))

<a id='pre'></a>
### Preprocessing
- remove missing values
- transform `MSSubClass` and `MoSold` to a categorical feature
- drop 'unimportant' categorical features
- preprocess categorical features
- split all_data into train and test again
- transform numerical features
    - log transform `SalePrice`
    - drop numerical features that have a correlation below 0.3
    - drop outliers
- Identify columns to drop for test set

In [4]:
train_pre = train.copy()
test_pre = test.copy()

#### remove missing values
> For this I will concat the train and test set. Since I am doing only basic missing value imputation (mean, median, and mode), the problem of data leakage will be minimized. When using more advanced methods like KNN-imputation or missForest pone should definitely perform those one the dataset seperately since information will leak into to train set from the test set. 

In [5]:
all_data = pd.concat([train_pre, test_pre], ignore_index=True)

In [6]:
# a lot of the missing values are just encodings for the instance that a specific feaure isn't available
# list of features with worng encodign for NA
feature_NA = ['Alley', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 
              'MiscFeature', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

# assign NA to None to indicate the lack of a certain feature
all_data[feature_NA] = all_data[feature_NA].fillna('None')

In [7]:
# imute missing categorical features mostly with the mode
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data['Utilities'] = all_data['Utilities'].fillna(all_data['Utilities'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Functional'] = all_data['Functional'].fillna(all_data['Functional'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

In [8]:
# imput missing numerical features (most numercial had only 1-2 missing values in that case I just imputet 0)
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(0)
all_data['BsmtFinSF1'] = all_data['BsmtFinSF1'].fillna(0)
all_data['BsmtFinSF2'] = all_data['BsmtFinSF2'].fillna(0)
all_data['BsmtUnfSF'] = all_data['BsmtUnfSF'].fillna(0)
all_data['TotalBsmtSF'] = all_data['TotalBsmtSF'].fillna(0)
all_data['BsmtFullBath'] = all_data['BsmtFullBath'].fillna(0)
all_data['BsmtHalfBath'] = all_data['BsmtHalfBath'].fillna(0)
all_data['GarageCars'] = all_data['GarageCars'].fillna(0)
all_data['GarageArea'] = all_data['GarageArea'].fillna(0)
all_data['GarageYrBlt'] = all_data['GarageYrBlt'].fillna(0)
# Neighorhood should impact the size of of street connected to the property
# code from https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

#### transform `MSSubClass` and `MoSold` to a categorical feature

In [9]:
all_data['MSSubClass'] = all_data['MSSubClass'].astype('str')

# create dict to assign categories
MoSold_dict = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
all_data['MoSold'] = all_data['MoSold'].map(MoSold_dict)

#### Cell to define which categorical features should be included 

In [10]:
# create DataFrame with relevant categorical feats
all_cat_feats = list(all_data.select_dtypes('object'))
# do the same for the numerical features befor processing categorical features and creating dummies
all_num_feats = list(all_data.select_dtypes(exclude='object'))

important_cat_feats = ['Neighborhood', 'MSZoning', 'MSSubClass', 'HouseStyle', 'Foundation', 'ExterQual', 'KitchenQual', 
                       'GarageQual', 'CentralAir', 'HeatingQC', 'BsmtQual']

cat_feats_drop = list(set(all_cat_feats) - set(important_cat_feats))

all_data.drop(columns=cat_feats_drop, inplace=True)

#### preprocess categorical features

In [11]:
# greate dummies for real categorical features with drop_first=True to reduce multicollinearity
list_dummies = ['MSSubClass', 'MSZoning', 'Neighborhood', 'HouseStyle', 'Foundation']
df_dummy = pd.get_dummies(all_data[list_dummies], drop_first=True)
all_data.drop(columns=list_dummies, inplace=True)
all_data = pd.concat([all_data, df_dummy], axis=1)

# converte categorical to ordinal features and use them as numeric input in the model
list_ord = ['ExterQual', 'KitchenQual', 'GarageQual', 'HeatingQC', 'BsmtQual']
map_dict_ord = {'None': 0, 'Po': 1, 'Fa': 2, 'TA':3, 'Gd':4, 'Ex': 5}
for ord_ in list_ord:
    all_data[ord_] = all_data[ord_].map(map_dict_ord)

# convert categorical to binary
all_data['CentralAir'] = all_data['CentralAir'].map({'Y': 1, 'N': 0})

#### Feature Engineering
- total square feet should be important when predicting the house price

In [12]:
# all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

#### Split all_data into train and test again

In [13]:
# split all_data in train and test to perform more preprocessing and feature engineering speratly (prevent data leakage)
train_pre = all_data.loc[:train.shape[0]-1]
test_pre = all_data.loc[train.shape[0]:]

In [14]:
test_pre.reset_index(drop=True, inplace=True)

In [15]:
train_pre.shape, test_pre.shape

((1460, 97), (1459, 97))

#### numerical features

##### Log transform `SalePrice` and save in target

In [16]:
target = np.log(train_pre.SalePrice)

##### drop numerical features that have a correlation below 0.2

In [17]:
# create matrix
corr_matrix = train_pre[all_num_feats]
corr_matrix = corr_matrix.corr()
corr_matrix = corr_matrix['SalePrice']

# filter matrix: drop num features with corrleation below 0.3
num_feats_drop = corr_matrix[corr_matrix < 0.2].index
train_pre.drop(columns=num_feats_drop, inplace=True)

##### drop outliers

In [18]:
train_pre = train_pre.drop(train_pre[(train_pre['GrLivArea']>4000) & (train_pre['SalePrice']<300000)].index)

##### add target to train

In [19]:
train_pre['SalePrice_log'] = target

#### Identify columns to drop for test set

In [20]:
# save id 
Id = test_pre['Id']

In [21]:
# features to drop
to_drop = list(set(test_pre) - set(train_pre))

In [22]:
test_pre.drop(columns=to_drop, inplace=True)

In [23]:
test_pre.drop(columns='SalePrice', inplace=True)

<a id='modeling'></a>
### Modeling
- make x_train and y_train 
- define Cross Validation
- initiate Models

#### make x_train and y_train 

In [24]:
# use train set to make x_train and y_train
x_train = train_pre.drop(columns=['SalePrice_log', 'SalePrice'])
y_train = train_pre['SalePrice_log']

In [25]:
x_train.shape, y_train.shape, test_pre.shape

((1458, 83), (1458,), (1459, 83))

In [26]:
# check if x_train and test_pre are identical
list(set(x_train) - set(test_pre)), list(set(test_pre) - set(x_train))

([], [])

#### Cross Validation
> define cross validation method to evaluate different machine learning models on the training data before using a model on the test data.

In [27]:
# code: https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmsle= np.sqrt(-cross_val_score(model, x_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmsle)

#### Models

In [28]:
# XGBoost
model_xgb = XGBRegressor()
# LightGBM
model_lgb = LGBMRegressor()
# CatBoost
model_cat = CatBoostRegressor(verbose=False)
# Lasso Regression
model_lasso = Lasso()
# Ridge Regression
model_ridge = Ridge()



# list of models for cross validation
models_list = [model_xgb, model_lgb, model_cat, model_lasso, model_ridge]
model_names = ['model_xgb', 'model_lgb', 'model_cat', 'model_lasso', 'model_ridge']

In [29]:
for model, name in zip(models_list, model_names):
    print(name + ' rmsle score:')
    print(np.mean(rmsle_cv(model)))
    print('#'*30)

model_xgb rmsle score:
0.1436501915261411
##############################
model_lgb rmsle score:
0.1349507619433274
##############################
model_cat rmsle score:
0.12534051694673637
##############################
model_lasso rmsle score:
0.16829118978283888
##############################
model_ridge rmsle score:
0.12196660786631104
##############################


### Use models to make predictions
- check the mean absolute error of each model
- use single model
- basic blended model

In [35]:
# split the data to evaluate each model
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(x_train, y_train, test_size=0.33, random_state=42)

for model, name in zip(models_list, model_names):
    model.fit(X_train_s, y_train_s)
    preds = model.predict(X_test_s)
    print(name + ' mean absolut error:')
    print(metrics.mean_absolute_error(np.exp(y_test_s), np.exp(preds)))

model_xgb mean absolut error:
17200.763550311203
model_lgb mean absolut error:
16550.233140546134
model_cat mean absolut error:
14852.295345761135
model_lasso mean absolut error:
21081.394337767597
model_ridge mean absolut error:
15000.05024620343


> the mean absolute error of each model is around 15000 to 20000, that means that the prediction of the model is around 15000 to 20000 from the real sale price

In [41]:
model_lgb.fit(X_train_s, y_train_s)
model_ridge.fit(X_train_s, y_train_s)
model_cat.fit(X_train_s, y_train_s)

# xgb_preds =model_xgb.predict(test_pre)
lgb_preds = model_lgb.predict(X_test_s)
ridge_preds = model_ridge.predict(X_test_s)
cat_preds = model_cat.predict(X_test_s)

preds = (lgb_preds + ridge_preds + cat_preds)/3

print(f'mean absolute error of the blened method: {metrics.mean_absolute_error(np.exp(y_test_s), np.exp(preds))}')

mean absolute error of the blened method: 14657.154068785994


> the final model (up to this day) can predict the house sale price with a prediction that is on the mean 145657 dollars of the real sale price (only based on the train set)

#### use LightGBM to make a submission
> used the following code to make predictions for multiple algorithms

In [30]:
# train on train data
model_lgb.fit(x_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [31]:
lgb_preds = model_lgb.predict(test_pre)

In [32]:
submission = pd.DataFrame()
submission['Id'] = Id
# transform log of SalePrice
submission['SalePrice'] = np.exp(lgb_preds)

In [33]:
submission.to_csv('sub_lgb_basis.csv', index=False)

In [34]:
submission

,Id,SalePrice
0,1461,116432.185670
1,1462,164286.503329
2,1463,178339.773554
3,1464,182750.515072
4,1465,196009.478068
...,...,...
1454,2915,72843.666095
1455,2916,82528.469590
1456,2917,166777.342132
1457,2918,109135.897066


#### Basic Model Blending
> use XGBRegressor, LGBMRegressor and Ridge Regression to make a prediction, and calculate the mean

In [35]:
# model_xgb.fit(x_train, y_train)
model_lgb.fit(x_train, y_train)
model_ridge.fit(x_train, y_train)
model_cat.fit(x_train, y_train)

In [36]:
# xgb_preds =model_xgb.predict(test_pre)
lgb_preds = model_lgb.predict(test_pre)
ridge_preds = model_ridge.predict(test_pre)
cat_preds = model_cat.predict(test_pre)

In [37]:
preds = (lgb_preds + ridge_preds + cat_preds)/3

In [38]:
submission = pd.DataFrame()
submission['Id'] = Id
# transform log of SalePrice
submission['SalePrice'] = np.exp(preds)

In [39]:
submission.to_csv('blended_model_basis1.csv', index=False)

In [40]:
submission

,Id,SalePrice
0,1461,119147.926766
1,1462,163339.278479
2,1463,179886.042188
3,1464,189303.193779
4,1465,194824.591609
...,...,...
1454,2915,74246.370232
1455,2916,83536.109919
1456,2917,172377.871315
1457,2918,110521.158742


### Hyperparameter tuning
> use GridSearch to find the best values for the parameter 

#### Ridge Regression
- alpha: Regularization strength
- fit_intercept: whether to fit the intercept for this model
- solver: for the computational routines

In [41]:
params_Ridge = {'alpha': [5, 1, 0.1], 
                'fit_intercept': [True, False],
                'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg']}

In [42]:
model_ridge = Ridge()
Ridge_gs = GridSearchCV(model_ridge, params_Ridge, scoring='neg_mean_squared_log_error', n_jobs=-2, cv=5, verbose=1)
Ridge_gs.fit(x_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  96 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-2)]: Done 120 out of 120 | elapsed:    4.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=-2,
             param_grid={'alpha': [5, 1, 0.1], 'fit_intercept': [True, False],
                         'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_log_error', verbose=1)

In [43]:
Ridge_gs.best_params_

{'alpha': 1, 'fit_intercept': True, 'solver': 'svd'}

#### LightGBM
- boosting_type: choose Gradient Boosting method
- num_leaves: maximum tree leaves for base learners (higher numbers can cause overfitting)
- learning_rate: learning rate of Boosting algo
- n_estimators: number of boosted trees

In [44]:
params_lgb = {'boosting_type': ['gbdt', 'dart'],
              'num_leaves': [31, 40, 50],
              'learning_rate': [0.1, 0.01],
              'n_estimators': [100, 150]}

In [45]:
modl_lgb = LGBMRegressor()
LightGBM_gs = GridSearchCV(model_lgb, params_lgb, scoring='neg_mean_squared_log_error', n_jobs=-2, cv=5, verbose=1)
LightGBM_gs.fit(x_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-2)]: Done 120 out of 120 | elapsed:   22.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=-2,
             param_grid={'boosting_type': ['gbdt', 'dart'],
                         'learning_rate': [0.1, 0.01],
                         'n_estimators': [100, 150],
    

In [46]:
LightGBM_gs.best_params_

{'boosting_type': 'gbdt',
 'learning_rate': 0.1,
 'n_estimators': 100,
 'num_leaves': 31}

#### CatBoost
- iterations: maximum number of trees that can be build
- learning_rate: specifies the gradient step
- l2_leaf_reg: coefficients for the regularization 

In [47]:
params_cat ={'iterations': [50, 100, 150],
             'learning_rate': [0.1, 0.05, 0.01],
             'l2_leaf_reg': [1, 5, 9]}

In [48]:
model_cat = CatBoostRegressor(verbose=False)
cat_gs = GridSearchCV(model_cat, params_cat, scoring='neg_mean_squared_log_error', n_jobs=-2, cv=5, verbose=1)
cat_gs.fit(x_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-2)]: Done 135 out of 135 | elapsed:   58.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostRegressor object at 0x7ffc5c3f67d0>,
             iid='warn', n_jobs=-2,
             param_grid={'iterations': [50, 100, 150], 'l2_leaf_reg': [1, 5, 9],
                         'learning_rate': [0.1, 0.05, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_log_error', verbose=1)

In [49]:
cat_gs.best_params_

{'iterations': 150, 'l2_leaf_reg': 1, 'learning_rate': 0.1}

#### Check tuned models 

In [50]:
# define models
model_ridge = Ridge(alpha=1, fit_intercept=True, solver='svd')
model_lgb = LGBMRegressor(booting_type='gbdt', learning_rate=0.1, n_estimators=100, num_leaves=31)
model_cat = CatBoostRegressor(iterations=150, l2_leaf_reg=1, learning_rate=0.1, verbose=False)

# make list for name and model
models_list = [model_lgb, model_cat, model_ridge]
model_names = ['model_lgb', 'model_cat', 'model_ridge']

# print result
for model, name in zip(models_list, model_names):
    print(name + ' rmsle score:')
    print(np.mean(rmsle_cv(model)))
    print('#'*30)

model_lgb rmsle score:
0.13562651988770133
##############################
model_cat rmsle score:
0.12674627552051065
##############################
model_ridge rmsle score:
0.12196660786631104
##############################
